In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import cv2 
import matplotlib.pyplot as plt

import os
import numpy as np
import pandas as pd
def show(img):
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [31]:
current_dir = os.getcwd()
img_list = [img for img in os.listdir(current_dir + '/sample/img') if img.endswith('.png')]
img_list = pd.DataFrame(img_list, columns=['img'])

In [ ]:
img = cv2.imread(current_dir + '/sample/img/' + img_list['img'][0])
show(img)

In [ ]:
# 이미지 크기 확인
height, width, _ = img.shape

overlap_ratio = 0.2
crop_size = 256
stride = int(crop_size * (1 - overlap_ratio))

imgs = []
for y in range(0, height, stride):
    for x in range(0, width, stride):
        # 이미지를 자를 때 마지막 블록을 강제로 크기 조정
        if x + crop_size > width:
            x = width - crop_size
        if y + crop_size > height:
            y = height - crop_size
        
        img_crop = img[y:y+crop_size, x:x+crop_size]

        # 크기가 crop_size보다 작더라도 마지막 블록은 강제 자르기
        imgs.append(img_crop)
        show(img_crop)


In [ ]:
resize = 256
resize_img = cv2.resize(img, (resize, resize))
show(resize_img)

In [ ]:
load = "./sample.png"
sample = cv2.imread(load)
x = 2700
y = 4700
img_crop = sample[x:x+crop_size, y:y+crop_size]
show(img_crop)

In [ ]:
def adjust_boxes_for_original_image(boxes, x_offset, y_offset):
    """
    슬라이싱된 이미지에서 예측된 바운딩 박스를 원본 이미지 좌표로 변환.
    """
    adjusted_boxes = []
    for box in boxes:
        x_min, y_min, x_max, y_max = box
        adjusted_box = [
            x_min + x_offset, y_min + y_offset,
            x_max + x_offset, y_max + y_offset
        ]
        adjusted_boxes.append(adjusted_box)
    return adjusted_boxes


In [2]:
from ultralytics import YOLO
import aifactory.score as aif
import time

In [ ]:
df = pd.DataFrame(columns=['image_name', 'cx', 'cy', 'w', 'h','r'])
crop_size = 512

class result:
    def __init__(self, cx, cy, w, h, r, conf):
        
        self.cx = cx
        self.cy = cy
        self.w = w
        self.h = h
        self.r = r
        self.conf = conf
    
    def boxes(self):
        return [self.cx, self.cy, self.w, self.h, self.r]
    
    def score(self):
        return self.conf

x = []    
    